In [2]:
# A simple graph example
import numpy as np
A = np.matrix([
    [0, 1, 0, 0],
    [0, 0, 1, 1],
    [0, 1, 0, 0],
    [1, 0, 1, 0]],
    dtype=float
)

In [3]:
# We generate 2 integer features for every node 
x = np.matrix([
    [i, -i]
    for i in range(A.shape[0])
], dtype=float)
x

matrix([[ 0.,  0.],
        [ 1., -1.],
        [ 2., -2.],
        [ 3., -3.]])

In [4]:
# Applying the propagation rule
# We have a graph, its adjacency matrix A and a set of input features x.
# Let apply the propagation rule:

In [6]:
conv = A*x
conv

matrix([[ 1., -1.],
        [ 5., -5.],
        [ 1., -1.],
        [ 2., -2.]])

In [7]:
#the representation of each (each row) is sum of its neighbors features

## Problem 

The aggregated representation of a node does not include its own features.
The representation is an aggregate of the features of neighbor nodes, so only nodes that 
has a self loop will include their own features in the aggregate

Nodes with large degrees will have large values in the feature representation while nodes
with small degrees will have small values. This can cause vanishing or exploding gradients. 
But is also problematic for stochastic gradient algorithms which are typically used to train such networks and are sensitive to the scale of each of the input features.

In [8]:
## Adding a self loops
# In practice by adding the identity matrix I to the adjacency matrix A before appling 
# propagation rule.

In [10]:
I = np.matrix(np.eye(A.shape[0]))
I

matrix([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [11]:
A_hat = A + I
conv = A_hat*x
conv

matrix([[ 1., -1.],
        [ 6., -6.],
        [ 3., -3.],
        [ 5., -5.]])

### Normalizing the feature representations

The feature representations can be normalized by node degree by transforming the adjacency matrix A by multiplying it with the inverse degree matrix D

F(X, A) = D^(-1)AX

In [15]:
D = np.array(np.sum(A, axis=0))[0]
print(D)
D = np.matrix(np.diag(D))
D

[1. 2. 2. 1.]


matrix([[1., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 2., 0.],
        [0., 0., 0., 1.]])

In [16]:
# before Adjacency matrix

A = np.matrix([
    [0, 1, 0, 0],
    [0, 0, 1, 1],
    [0, 1, 0, 0],
    [1, 0, 1, 0]],
    dtype=float
)

In [18]:
# After normalization

normalized = D**-1 * A
normalized

matrix([[0. , 1. , 0. , 0. ],
        [0. , 0. , 0.5, 0.5],
        [0. , 0.5, 0. , 0. ],
        [1. , 0. , 1. , 0. ]])

In [19]:
## observe that the weights in each row of the adjacency matrix have been
# divided by the degree of the node corresponding to the row.

In [20]:
conv = D**-1 * A * x
conv

matrix([[ 1. , -1. ],
        [ 2.5, -2.5],
        [ 0.5, -0.5],
        [ 2. , -2. ]])

## Putting it all together
We now combine the self loop and normalization tips

In [21]:
# adding back the weights
# first order of business is applying the weights. Note that here D_hat is the degree
# matrix of A_hat = A + I, the degree matrix of A with forced self loops

In [24]:
W = np.matrix([
    [1, -1],
    [-1, 1]
])
W # shape of weight matrix is 2 by 2 because input features are two.

matrix([[ 1, -1],
        [-1,  1]])

In [27]:
D_hat = np.array(np.sum(A_hat, axis=0))[0]

D_hat = np.matrix(np.diag(D_hat))
D_hat

matrix([[2., 0., 0., 0.],
        [0., 3., 0., 0.],
        [0., 0., 3., 0.],
        [0., 0., 0., 2.]])

In [28]:
final_conv = D_hat ** -1 *A_hat * x * W
final_conv

matrix([[ 1., -1.],
        [ 4., -4.],
        [ 2., -2.],
        [ 5., -5.]])

In [29]:
# And if we want to reduce the dimensionality of the output feature
# representations we can reduce the size of the weight matrix w:

In [30]:
W = np.matrix([
    [1],
    [-1]
])

In [33]:
new_conv = D_hat ** -1 * A_hat * x * W
new_conv

matrix([[1.],
        [4.],
        [2.],
        [5.]])

In [34]:
# adding an activation fucntion
# we choose to preserve the dimensionality of the feature representations
# and apply the ReLU activation function

In [62]:
import torch.nn as nn
import torch
W = np.matrix([
    [1, -1],
    [-1, 1]
])

conv = D_hat**-2 * A_hat * x * W
print(conv)

conv1 = torch.from_numpy(conv)
conv2 = torch.nn.functional.relu(conv1)


[[ 0.5        -0.5       ]
 [ 1.33333333 -1.33333333]
 [ 0.66666667 -0.66666667]
 [ 2.5        -2.5       ]]


In [63]:
conv2

tensor([[0.5000, 0.0000],
        [1.3333, 0.0000],
        [0.6667, 0.0000],
        [2.5000, 0.0000]], dtype=torch.float64)

## Zachary's Karate Club

It is a commonly used social network where nodes represent members of a karate club and the edges their mutual relations. While Zachary was studying the karate club, a conflict arose between the administrator and the instructor which resulted in the club splitting in two. The figure below shows the graph representation of the network and nodes are labeled according to which part of the club. The administrator and instructor are marked with A and I respectively.

In [64]:
# Building the GCN


In [65]:
from networkx import karate_club_graph, to_numpy_matrix

In [66]:
zkc = karate_club_graph()
order = sorted(list(zkc.nodes()))



In [68]:
A = to_numpy_matrix(zkc, nodelist=order)
I = np.eye(zkc.number_of_nodes())

In [70]:
A.shape

(34, 34)

In [71]:
A

matrix([[0., 1., 1., ..., 1., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 1., 0.],
        ...,
        [1., 0., 0., ..., 0., 1., 1.],
        [0., 0., 1., ..., 1., 0., 1.],
        [0., 0., 0., ..., 1., 1., 0.]])

In [72]:
I.shape

(34, 34)

In [73]:
I

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [74]:
A_hat = A + I

In [75]:
D_hat = np.array(np.sum(A_hat, axis=0))[0]
D_hat = np.matrix(np.diag(D_hat))
D_hat

matrix([[17.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., 10.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0., 11., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  7.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0., 13.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0., 18.]])

In [77]:
# now initialize weight randomly
W_1 = np.random.normal(
    loc=0, scale=1, size=(zkc.number_of_nodes(), 4))

W_2 = np.random.normal(
    loc=0, size=(W_1.shape[1], 2))

In [78]:
# Stack the GCN layers. We here use just the identity matrix as feature representation,
# that is, each node is represented as a one-hot encoded  categorical variable.

In [82]:
import torch
def gcn_layer(A_hat, D_hat, X, W):
    conv = D_hat**-1 * A_hat * X * W
    conv1 = torch.from_numpy(conv)
    conv1 = torch.nn.functional.relu(conv1)
    return conv1

H_1 = gcn_layer(A_hat, D_hat, I, W_1)
H_1 = torch.Tensor.numpy(H_1)
H_2 = gcn_layer(A_hat, D_hat, H_1, W_2)

output = H_2

In [83]:
feature_representations = {
    node: np.array(output)[node]
    for node in zkc.nodes()
}

In [84]:
feature_representations

{0: array([0.        , 0.04670342]),
 1: array([0.       , 0.0594083]),
 2: array([0.        , 0.11668517]),
 3: array([0.        , 0.07472222]),
 4: array([0.        , 0.00941296]),
 5: array([0.        , 0.00786711]),
 6: array([0.        , 0.00753037]),
 7: array([0.        , 0.09704341]),
 8: array([0.        , 0.10349355]),
 9: array([0.        , 0.13489534]),
 10: array([0.        , 0.00952861]),
 11: array([0.        , 0.00695494]),
 12: array([0.        , 0.04918243]),
 13: array([0.        , 0.05351715]),
 14: array([0.        , 0.04520564]),
 15: array([0.      , 0.220111]),
 16: array([0., 0.]),
 17: array([0.        , 0.00840705]),
 18: array([0.        , 0.05549262]),
 19: array([0.       , 0.0316059]),
 20: array([0.        , 0.15016782]),
 21: array([0.        , 0.01031932]),
 22: array([0.        , 0.18676205]),
 23: array([0.        , 0.14602732]),
 24: array([0.        , 0.18811123]),
 25: array([0.        , 0.17650049]),
 26: array([0.        , 0.17067942]),
 27: arr